In [1]:
from os.path import join
import pandas as pd
idx = pd.IndexSlice

In [2]:
current_quarter_num = 4
current_quarter = 'Q{}'.format(current_quarter_num)
current_year = 2017
prev_year = current_year - 1
quarter_word = 'fourth'
current_y_q = '{} {}'.format(current_year, current_quarter)
prev_y_q = '{} {}'.format(prev_year, current_quarter)
roll_y_q = '{} Q{}'.format(prev_year, current_quarter_num - 1)

index_2005 = 599.1
index_2005_lb = 1321

## Import quarterly index and generation results

In [3]:
index = pd.read_csv('Quarterly index {} {}.csv'.format(current_year, current_quarter))
index.set_index('year_quarter', inplace=True)
gen = pd.read_csv('Quarterly generation {} {}.csv'.format(current_year, current_quarter))

total_gen = gen.groupby(['year', 'quarter'], as_index=False).sum()
total_gen['year_quarter'] = total_gen.apply(lambda row: '{} Q{}'
                                            .format(int(row['year']), 
                                                    int(row['quarter'])), axis=1)
total_gen['fuel category'] = 'Total'

gen = pd.concat([gen, total_gen])
fuel_cats = gen['fuel category'].unique()
gen.set_index(['fuel category', 'year_quarter'], inplace=True)


## Compare to previous quarter

Comparing Q3 2017 to Q3 2016
The Carnegie Mellon Power Sector Carbon Index provides a comprehensive picture of the carbon intensity of electricity production in the during the previous 12 months and over an extended period to 2001. The CMU index also provides a summary of how much electricity generation is from coal, natural gas, nuclear, and renewables. According to the Carnegie Mellon Power Sector Carbon Index, U.S. power plant emissions averaged 1,035 lb CO2 per MWh in the third quarter of 2017, which was down 5 percent from the same time frame in 2016. Continuing the trend seen in Q2 2017 and the first three quarters of 2016, more electricity was generated from natural gas than from coal. Electricity generation from renewables was 10% higher in the third quarter of 2017 when compared to the third quarter of 2016.

In [4]:
index.tail()

,year,quarter,generation (mwh),final co2 (kg),index (g/kwh),change since 2005,index (lb/mwh)
year_quarter,,,,,,,
2016 Q4,2016,4,9.592564e+08,4.320641e+11,450.415699,-0.248218,992.986449
2017 Q1,2017,1,9.560930e+08,4.077586e+11,426.484278,-0.288161,940.227240
2017 Q2,2017,2,9.791486e+08,4.162425e+11,425.106560,-0.290461,937.189922
2017 Q3,2017,3,1.123963e+09,5.243688e+11,466.535480,-0.221312,1028.524119
2017 Q4,2017,4,9.753934e+08,4.214174e+11,432.048656,-0.278874,952.494466


In [5]:
curr_q_index_lb = index.loc[current_y_q, 'index (lb/mwh)']
curr_q_index_str = '{:,}'.format(int(round(curr_q_index_lb)))

prev_q_index_lb = index.loc[prev_y_q, 'index (lb/mwh)']
prev_q_index_str = '{:,}'.format(int(round(prev_q_index_lb)))

index_yoy_change = curr_q_index_lb - prev_q_index_lb
index_yoy_per = index_yoy_change / prev_q_index_lb
index_yoy_per_str = '{:.0f}'.format(abs(index_yoy_per * 100))

if index_yoy_per < 0:
    direction = 'down'
else:
    direction = 'up'

In [6]:
base_str_prev_q = 'The Carnegie Mellon Power Sector Carbon Index provides a comprehensive picture of the carbon intensity of electricity production in the during the previous 12 months and over an extended period to 2001. The CMU index also provides a summary of how much electricity generation is from coal, natural gas, nuclear, and renewables. According to the Carnegie Mellon Power Sector Carbon Index, U.S. power plant emissions averaged {} lb CO<sub>2</sub> per MWh in the {} quarter of {}, which was {} {} percent from the same time frame in {}.'
prev_q_str = base_str_prev_q.format(curr_q_index_str, quarter_word, current_year, direction, index_yoy_per_str, str(prev_year))

In [7]:
prev_q_str

'The Carnegie Mellon Power Sector Carbon Index provides a comprehensive picture of the carbon intensity of electricity production in the during the previous 12 months and over an extended period to 2001. The CMU index also provides a summary of how much electricity generation is from coal, natural gas, nuclear, and renewables. According to the Carnegie Mellon Power Sector Carbon Index, U.S. power plant emissions averaged 952 lb CO<sub>2</sub> per MWh in the fourth quarter of 2017, which was down 4 percent from the same time frame in 2016.'

## Compare to 2005

Comparing Performance in 2017 to 2005
The Power Sector Carbon Index for Q3 of 2017 was 22% lower than in the annual value in 2005, from 1,324 lb CO2 per MWh to 1,004 lb CO2 per MWh (or 600 to 470 g CO2 per kWh). As of Q3 of 2017, the rolling annual average carbon intensity was 981 lb CO2 per MWh (or 445 g CO2 per kWh), which is 26% lower than the annual value in 2005.

In [8]:
curr_q_index_lb = round(index.loc[current_y_q, 'index (lb/mwh)'])
curr_q_index = index.loc[current_y_q, 'index (g/kwh)']
curr_roll_index = (index.loc[roll_y_q:, 'final co2 (kg)'].sum()
                   / index.loc[roll_y_q:, 'generation (mwh)'].sum())
curr_roll_index_lb = round(curr_roll_index * 2.2046)
index_change_2005_str = '{:.0%}'.format(abs(index.loc[current_y_q, 'change since 2005']))
roll_change_2005 = abs((index_2005 - curr_roll_index) / index_2005)
roll_change_2005_str = '{:.0%}'.format(roll_change_2005)


In [9]:
base_str_2005 = 'The Power Sector Carbon Index for {} of {} was {} lower than in the annual value in 2005, from {:,} lb CO<sub>2</sub> per MWh to {:,.0f} lb CO<sub>2</sub> per MWh (or {:.0f} to {:.0f} g CO<sub>2</sub> per kWh). As of {} of {}, the rolling annual average carbon intensity was {:,.0f} lb CO<sub>2</sub> per MWh (or {:.0f} g CO<sub>2</sub> per kWh), which is {} lower than the annual value in 2005.'
change_2005_str = base_str_2005.format(current_quarter, current_year,
                                       index_change_2005_str, index_2005_lb,
                                       curr_q_index_lb, index_2005, curr_q_index,
                                       current_quarter, current_year, curr_roll_index_lb,
                                       curr_roll_index, roll_change_2005_str)

In [10]:
change_2005_str

'The Power Sector Carbon Index for Q4 of 2017 was 28% lower than in the annual value in 2005, from 1,321 lb CO<sub>2</sub> per MWh to 952 lb CO<sub>2</sub> per MWh (or 599 to 432 g CO<sub>2</sub> per kWh). As of Q4 of 2017, the rolling annual average carbon intensity was 995 lb CO<sub>2</sub> per MWh (or 451 g CO<sub>2</sub> per kWh), which is 25% lower than the annual value in 2005.'

### Generation stats

In [11]:
fuels = ['Total', 'Coal', 'Hydro', 'Natural Gas', 'Wind', 'Solar', 
         'Nuclear', 'Other', 'Other Renewables', ]
gen_results = []
for fuel in fuels:
    prev_gen = gen.loc[idx[fuel, prev_y_q], 'generation (mwh)']
    prev_gen_str = '{:.1f} million MWh'.format(prev_gen / 1e6)
    
    curr_gen = gen.loc[idx[fuel, current_y_q], 'generation (mwh)']
    curr_gen_str = '{:.1f} million MWh'.format(curr_gen / 1e6)
    
    per_change = (curr_gen - prev_gen) / prev_gen
    per_change_str = '{:.0%}'.format(abs(per_change))
    
    if prev_gen > curr_gen:
        direction = 'down'
    else:
        direction = 'up'
    
    kwargs = {
        'fuel': fuel,
        'dir': direction,
        'change': per_change_str,
        'curr_q': current_y_q,
        'curr_gen': curr_gen_str,
        'prev_q': prev_y_q,
        'prev_gen': prev_gen_str
    }
    
    result_str = '- {fuel} generation was {dir} {change} in {curr_q} ({curr_gen}) when compared to {prev_q} ({prev_gen}).'.format(**kwargs)
    
    if fuel != 'Total':
        total_gen = gen.loc[idx['Total', current_y_q], 'generation (mwh)']
        per_gen = curr_gen / total_gen
        per_gen_str = '{:.0%}'.format(per_gen)
        
        second_str = ' {} represented {} of total generation in {}'.format(fuel, per_gen_str, current_y_q)
        
        result_str += second_str
    
    gen_results.append(result_str)

In [12]:
gen_results

['- Total generation was up 2% in 2017 Q4 (975.4 million MWh) when compared to 2016 Q4 (959.3 million MWh).',
 '- Coal generation was down -6% in 2017 Q4 (287.7 million MWh) when compared to 2016 Q4 (304.9 million MWh). Coal represented 29% of total generation in 2017 Q4',
 '- Hydro generation was up 1% in 2017 Q4 (59.5 million MWh) when compared to 2016 Q4 (58.7 million MWh). Hydro represented 6% of total generation in 2017 Q4',
 '- Natural Gas generation was up 4% in 2017 Q4 (304.5 million MWh) when compared to 2016 Q4 (293.2 million MWh). Natural Gas represented 31% of total generation in 2017 Q4',
 '- Wind generation was up 13% in 2017 Q4 (70.9 million MWh) when compared to 2016 Q4 (62.9 million MWh). Wind represented 7% of total generation in 2017 Q4',
 '- Solar generation was up 30% in 2017 Q4 (16.0 million MWh) when compared to 2016 Q4 (12.3 million MWh). Solar represented 2% of total generation in 2017 Q4',
 '- Nuclear generation was up 4% in 2017 Q4 (206.3 million MWh) when co

## Write to file

In [13]:
outF = open("blog.txt", "w")
outF.write(prev_q_str)
outF.write('\n\n')
outF.write(change_2005_str)
outF.write('\n\n')

for line in gen_results:
    outF.write(line)
    outF.write('\n')
outF.close()